# VacationPy
### Data Visualization of weather variables and place (location) markers

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
## Load Cities.csv as DataFrame
cities_df = pd.read_csv('../WeatherPy/Output/Cities.csv')
## Display DataFrame preview
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saskylakh,71.9167,114.0833,34.02,99,100,7.47,RU,1619754043
1,Lebu,-37.6167,-73.6500,55.09,94,100,8.59,CL,1619754043
2,Polis,35.0367,32.4264,66.20,27,0,5.75,CY,1619754043
3,Mahébourg,-20.4081,57.7000,77.00,83,75,14.97,MU,1619754044
4,Princeton,25.5384,-80.4089,78.80,83,1,3.44,US,1619754044


### Humidity Heatmap
* Configure gmaps.
* Use Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
## Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
fig = gmaps.figure(center=(15,15), zoom_level=2.1)

locations = cities_df[['Lat', 'Lng']].values.tolist()
weights = cities_df['Humidity'].values.tolist()
max_humidity = cities_df['Humidity'].max()

heat_layer = gmaps.heatmap_layer(
    locations=locations,
    weights=weights,
    dissipating=False,
    max_intensity=max_humidity,
    point_radius = 3
)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
## Conditional selection of cities according to ideal weather criteria:
    ## 1. Max temperature lower than 80 degrees but higher than 70.
    ## 2. Wind speed less than 10 mph.
    ## 3. Zero cloudiness.
    
## Pandas series boolean (for .loc)
ideal_weather = (cities_df['Max Temp'] > 70) & \
                   (cities_df['Max Temp'] < 80) & \
                   (cities_df['Wind Speed'] < 10) & \
                   (cities_df['Cloudiness'] == 0)

## Print total vs. selected cities
print(f"Total number of cities: {len(cities_df)}")
print(f"Cities fitting ideal weather criteria: {sum(ideal_weather)}")

Total number of cities: 570
Cities fitting ideal weather criteria: 13


In [6]:
## Build ideal weather cities DataFrame
ideal_cities_df = cities_df[['City', 'Country', 'Lat', 'Lng']].loc[ideal_weather]
## Drop any rows will null values
ideal_cities_df = ideal_cities_df.dropna(axis=0, inplace=False)
## Reset index
ideal_cities_df = ideal_cities_df.reset_index(drop=True, inplace=False)

## Display DataFrame
ideal_cities_df

,City,Country,Lat,Lng
0,Broken Hill,AU,-31.9500,141.4333
1,Batemans Bay,AU,-35.7167,150.1833
2,Cabo San Lucas,MX,22.8909,-109.9124
3,Maţāy,EG,28.4190,30.7792
4,Olinda,BR,-8.0089,-34.8553
5,Korla,CN,41.7597,86.1469
6,Inhambane,MZ,-23.8650,35.3833
7,Altar,MX,30.7000,-111.8167
8,Teacapan,MX,22.5500,-105.7500
9,Suez,EG,29.9737,32.5263


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
## GOOGLE MAPS API

## Nearby Search URL
nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

## Nearby Search Parameters
params = {
    'key': g_key,
    'radius': 5000,
    'type': "lodging",
    'language': "en"
}

In [8]:
## Set initial values
hotel_name = []
hotel_lat = []
hotel_lng = []
failed_index = []

## Print Log Header
print("Beginning Data Retrieval")
print("------------------------------")

for index, row in ideal_cities_df.iterrows():
    ## Adjust search parameters for current city
    params['location'] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(nearby_url, params=params).json()
    results = response['results']
    ## Print Status
    print(f"Processing {row['City']}:")
    
    try:
        ## Print first hotel name from results
        print(f" - First hotel is {results[0]['name']}")
        ## Retrieve hotel data
        hotel_name.append(results[0]['name'])
        hotel_lat.append(results[0]['geometry']['location']['lat'])
        hotel_lng.append(results[0]['geometry']['location']['lng'])

    except (KeyError, IndexError):
        ## Store exception index
        failed_index.append(index)
        ## Print error message
        print(" - Missing field/result... skipping.")
        
## Print Log Footer
print("------------------------------")        
print("Data Retrieval Complete")
print("------------------------------")

Beginning Data Retrieval
------------------------------
Processing Broken Hill:
 - First hotel is Royal Exchange Hotel
Processing Batemans Bay:
 - First hotel is Batemans Bay Marina Resort
Processing Cabo San Lucas:
 - First hotel is Hotel Tesoro Los Cabos
Processing Maţāy:
 - First hotel is Nag Aespen
Processing Olinda:
 - First hotel is Hotel Albergue de Olinda
Processing Korla:
 - First hotel is Gulisitan Hotel
Processing Inhambane:
 - First hotel is Oceano
Processing Altar:
 - First hotel is Hotel Maria Elena
Processing Teacapan:
 - First hotel is Hotel los Mangos
Processing Suez:
 - First hotel is Green House
Processing Dubbo:
 - First hotel is Comfort Inn Blue Lagoon
Processing Beloha:
 - Missing field/result... skipping.
Processing Jinxiang:
 - First hotel is Wolong Village
------------------------------
Data Retrieval Complete
------------------------------


In [9]:
## Build Hotel DataFrame
hotel_df = ideal_cities_df[['City', 'Country']]
## Drop cities with missing field/result 
hotel_df = hotel_df.drop(index=failed_index, inplace=False)
## Reset index
hotel_df = hotel_df.reset_index(drop=True, inplace=False)
## Insert retrieved hotel data
hotel_df.insert(0, 'Hotel Name', hotel_name)
hotel_df.insert(len(hotel_df.columns), 'Lat', hotel_lat)
hotel_df.insert(len(hotel_df.columns), 'Lng', hotel_lng)

## Display Hotel DataFrame
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,Royal Exchange Hotel,Broken Hill,AU,-31.957779,141.465727
1,Batemans Bay Marina Resort,Batemans Bay,AU,-35.717871,150.190809
2,Hotel Tesoro Los Cabos,Cabo San Lucas,MX,22.880938,-109.911797
3,Nag Aespen,Maţāy,EG,28.423360,30.734995
4,Hotel Albergue de Olinda,Olinda,BR,-8.015274,-34.846542
5,Gulisitan Hotel,Korla,CN,41.753048,86.144840
6,Oceano,Inhambane,MZ,-23.861426,35.346690
7,Hotel Maria Elena,Altar,MX,30.715408,-111.836861
8,Hotel los Mangos,Teacapan,MX,22.546760,-105.741353
9,Green House,Suez,EG,29.960716,32.560655


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))